# 03_mtr_labor_plus_one_2024.ipynb

## Part A — What we are doing

We estimate the **rebate-only Marginal Tax Rate (MTR)** impact from the **phase-out** on labor income. For households with wages > 0, we:
1. Compute baseline rebate `R0`.
2. Add +$1 to **household AGI** (simulating +$1 of labor income), recompute rebate `R1`.
3. Set **MTR = −(R1 − R0)**.

We report:
- **Population-weighted** average MTR.
- **Earnings-weighted** average MTR (weights ∝ wages).
- **Shares of wages** inside the phase-out bands:
  - Singles: \$75k–\$125k
  - Married: \$150k–\$200k

**Outputs**
- `outputs/vat/mtr_summary_2024.csv`
- `outputs/vat/wage_phaseout_shares_2024.csv`

---

## Part B — Inputs & required fields

- **Reads:** `intermediate/ca_panel_2024.(parquet|csv)` from `01`.
- Fields needed:
  - `household_agi`, `employment_income`, `household_weight`
  - `filing_status` (to pick thresholds/bands)
  - Allowance + phase-out computation or functions to recompute them.

---

## Part C — Computation details

- **Baseline**: use `rebate_after_phaseout` from panel (or recompute to be safe).
- **Counterfactual**: `agi_plus_one = household_agi + 1.0` (only for those with wages > 0).
  - Re-run the **phase-out formula** holding allowance fixed (size/status unchanged).
  - Compute `R1` and then **MTR = −(R1 − R0)**.

**Band shares**
- For each status, flag households whose **AGI** lies in the band.
- Compute share of **total wages** and **share of total households** in-band.

---

## Part D — Deliverables & acceptance checks

**Files written**
- `outputs/vat/mtr_summary_2024.csv` (two rows: population-weighted, earnings-weighted)
- `outputs/vat/wage_phaseout_shares_2024.csv` (rows: Single band, Married band; columns: wage share, population share)

**Acceptance checks**
- MTR values are finite and plausible (typically small in absolute terms).
- Band shares lie in **[0,1]** and sum ≤ 1 within each status.

---

## Part E — Troubleshooting

- **NaNs in MTR**: ensure every row used has `employment_income > 0`, positive weights, and valid `filing_status`.
- **Extreme MTRs**: check that only **AGI** changes (by +$1). Allowance (size/status) must remain fixed.


In [11]:
# 03 — Rebate-only MTR on labor (+$1), 2024
# [dynamic bands; normalized weights; decomposition output]
import os, numpy as np, pandas as pd, importlib.util

print("Step 03 start.")

# -------------------------
# Load helpers
# -------------------------
vat_path = os.path.abspath("../policy/vat_rebate.py")
spec = importlib.util.spec_from_file_location("vat_rebate", vat_path)
vr = importlib.util.module_from_spec(spec); spec.loader.exec_module(vr)
print("Loaded:", vr.__file__)

# -------------------------
# Load panel from Step 01
# -------------------------
parq = "../intermediate/ca_panel_2024.parquet"
csv  = "../intermediate/ca_panel_2024.csv"
if os.path.exists(parq):
    try:
        df = pd.read_parquet(parq)
        panel_path = parq
    except Exception as e:
        print(f"[warn] Parquet read failed ({e}); falling back to CSV if present.")
        if os.path.exists(csv):
            df = pd.read_csv(csv)
            panel_path = csv
        else:
            raise
elif os.path.exists(csv):
    df = pd.read_csv(csv)
    panel_path = csv
else:
    raise FileNotFoundError("Missing panel; run Step 01 to create ca_panel_2024.(parquet|csv).")

print("Panel path:", panel_path)
print("Panel shape:", df.shape)

# -------------------------
# Normalize weight column
# -------------------------
if "weight" not in df.columns:
    w_aliases = [c for c in df.columns if c.lower() in ("household_weight","weight","hh_weight")]
    if not w_aliases:
        raise KeyError("No weight column found (looked for household_weight/weight/hh_weight).")
    df["weight"] = pd.to_numeric(df[w_aliases[0]], errors="coerce").fillna(0.0)
else:
    df["weight"] = pd.to_numeric(df["weight"], errors="coerce").fillna(0.0)

print(f"[diag] Weighted CA households (after Step 01 deflator): {df['weight'].sum():,.0f}")

# Ensure required structural columns
need = ["household_agi","employment_income","size_bucket","is_married_couple"]
missing = [c for c in need if c not in df.columns]
if missing:
    raise KeyError(f"Missing required columns: {missing}")

# Ensure allowance & base-phaseout present
if "consumption_allowance" not in df.columns:
    df = vr.compute_allowance(df)
if "rebate_after_phaseout" not in df.columns:
    df = vr.apply_phaseout(df)

# -------------------------
# Base and +$1 AGI scenarios
# -------------------------
base = df.copy()
plus = df.copy()

# +$1 AGI for households with wages > 0
has_wages = plus["employment_income"].fillna(0) > 0
plus.loc[has_wages, "household_agi"] = plus.loc[has_wages, "household_agi"] + 1.0

# Recompute rebate for 'plus' (allowance unchanged; phase-out depends on AGI & status)
plus["consumption_allowance"] = base["consumption_allowance"]
plus = vr.apply_phaseout(plus)

# Δrebate = rebate_plus - rebate_base → MTR (tax) = −Δrebate
d_rebate = plus["rebate_after_phaseout"] - base["rebate_after_phaseout"]
mtr_tax  = -d_rebate

# -------------------------
# Weighted averages
# -------------------------
w = df["weight"].astype(float)
wages = df["employment_income"].astype(float).clip(lower=0.0)
mask = has_wages.fillna(False)

pop_wtd  = float((mtr_tax[mask] * w[mask]).sum() / w[mask].sum()) if w[mask].sum() > 0 else 0.0
earn_wtd = float((mtr_tax[mask] * wages[mask]).sum() / wages[mask].sum()) if wages[mask].sum() > 0 else 0.0

# -------------------------
# Dynamic phase-out band shares 
# -------------------------
if "filing_status" in df.columns:
    fs = df["filing_status"].astype(str).str.lower()
else:
    fs = pd.Series(np.where(df["is_married_couple"].astype(int)==1, "mfj", "single"), index=df.index)

agi = df["household_agi"].astype(float)

thr = fs.map({"single": vr.THRESHOLDS["single"], "mfj": vr.THRESHOLDS["mfj"]}).astype(float)
rng = fs.map({"single": vr.PHASE_RANGE["single"], "mfj": vr.PHASE_RANGE["mfj"]}).astype(float)
band_start = thr
band_end   = thr + rng

single_band_mask = (fs.eq("single") & (agi >= band_start) & (agi <= band_end))
mfj_band_mask    = (fs.eq("mfj")    & (agi >= band_start) & (agi <= band_end))

wages_total = float(wages.sum())
share_single = float(wages[single_band_mask].sum() / wages_total) if wages_total > 0 else 0.0
share_mfj    = float(wages[mfj_band_mask].sum()    / wages_total) if wages_total > 0 else 0.0

single_label = f"{int(vr.THRESHOLDS['single']):,}–{int(vr.THRESHOLDS['single']+vr.PHASE_RANGE['single']):,}"
mfj_label    = f"{int(vr.THRESHOLDS['mfj']):,}–{int(vr.THRESHOLDS['mfj']+vr.PHASE_RANGE['mfj']):,}"

# -------------------------
# Save standard outputs
# -------------------------
os.makedirs("../outputs/vat", exist_ok=True)

pd.DataFrame([{
    "regime": "rebate_only",
    "year": 2024,
    "population_weighted_MTR": pop_wtd,
    "earnings_weighted_MTR":   earn_wtd,
}]).to_csv("../outputs/vat/mtr_summary_2024.csv", index=False)

pd.DataFrame([{
    "year": 2024,
    "single_band_label": single_label,
    "mfj_band_label":    mfj_label,
    "share_wages_single_in_band": share_single,
    "share_wages_mfj_in_band":    share_mfj,
}]).to_csv("../outputs/vat/wage_phaseout_shares_2024.csv", index=False)

# -------------------------
# Extra output: slopes & in-band shares by status × size
# -------------------------
in_band = (agi >= band_start) & (agi <= band_end)
allow   = df["consumption_allowance"].astype(float)
band_w  = rng
slope   = np.where(in_band, allow / band_w, 0.0)

size   = df["size_bucket"].astype(int)
stat   = fs

grp = pd.DataFrame({
    "status": stat,
    "size_bucket": size,
    "w": w,
    "w_inband": np.where(in_band, w, 0.0),
    "wages": wages,
    "wages_inband": np.where(in_band, wages, 0.0),
    "slope_inband_w": slope * np.where(in_band, w, 0.0),
    "slope_inband_wages": slope * np.where(in_band, wages, 0.0),
})

agg = grp.groupby(["status","size_bucket"], as_index=False).sum()
agg["pop_share_in_band"]   = np.where(agg["w"] > 0, agg["w_inband"] / agg["w"], np.nan)
agg["wage_share_in_band"]  = np.where(agg["wages"] > 0, agg["wages_inband"] / agg["wages"], np.nan)
agg["avg_slope_inband_pop"]  = np.where(agg["w_inband"] > 0, agg["slope_inband_w"] / agg["w_inband"], np.nan)
agg["avg_slope_inband_earn"] = np.where(agg["wages_inband"] > 0, agg["slope_inband_wages"] / agg["wages_inband"], np.nan)

status_order = pd.Categorical(agg["status"], categories=["single","mfj"], ordered=True)
size_order   = pd.Categorical(agg["size_bucket"], categories=[1,2,3,4,5,6,7], ordered=True)
agg = agg.assign(status=status_order, size_bucket=size_order).sort_values(["status","size_bucket"])

slopes_path = "../outputs/vat/mtr_slope_by_status_size_2024.csv"
agg.to_csv(slopes_path, index=False)

# -------------------------
# Checks & printouts
# -------------------------
assert np.isfinite([pop_wtd, earn_wtd]).all()
assert 0.0 <= share_single <= 1.0
assert 0.0 <= share_mfj    <= 1.0

print(f"[save] ../outputs/vat/mtr_summary_2024.csv")
print(f"[save] ../outputs/vat/wage_phaseout_shares_2024.csv")
print(f"[save] {slopes_path}")

print(f"✅ Step 03 done.")
print(f"   Rebate-only MTR: population-weighted={pop_wtd:.4f}, earnings-weighted={earn_wtd:.4f}")
print(f"   Wage shares in phase-out bands: Single ({single_label})={share_single:.2%}, "
      f"MFJ ({mfj_label})={share_mfj:.2%}")

print("\nDecomposition preview (shares as fractions):")
print(agg[[
    "status","size_bucket",
    "pop_share_in_band","wage_share_in_band",
    "avg_slope_inband_pop","avg_slope_inband_earn"
]].to_string(index=False))


Step 03 start.
Loaded: c:\Users\Ali.Melad\Dropbox\Ali Work\Kyle\California VAT\policy_engile_cali_v2\policy\vat_rebate.py
Panel path: ../intermediate/ca_panel_2024.csv
Panel shape: (1747, 15)
[diag] Weighted CA households (after Step 01 deflator): 14,431,591
[save] ../outputs/vat/mtr_summary_2024.csv
[save] ../outputs/vat/wage_phaseout_shares_2024.csv
[save] ../outputs/vat/mtr_slope_by_status_size_2024.csv
✅ Step 03 done.
   Rebate-only MTR: population-weighted=0.0754, earnings-weighted=0.1017
   Wage shares in phase-out bands: Single (50,000–100,000)=5.16%, MFJ (100,000–200,000)=21.72%

Decomposition preview (shares as fractions):
status size_bucket  pop_share_in_band  wage_share_in_band  avg_slope_inband_pop  avg_slope_inband_earn
single           1           0.079242            0.222823                0.2916                 0.2916
single           2           0.138320            0.116996                0.3944                 0.3944
single           3           0.198368            0.